In [1]:
import sys

import statsmodels.tsa.stattools as ts
import statsmodels.tsa.stattools as ts

#Financial data will be importing via Yahoo finance
import yfinance as yf
import scipy

import numpy as np

import datetime
from datetime import timedelta

In [39]:
#!{sys.executable} -m pip install datetime

In [3]:
def cointegrated_pairs(data):
    n =  (len(data.columns) -1)
    stocks = data.iloc[:,1:].columns.values #pulls all stock names 
    data = data.to_numpy()
    pvalue_matrix = np.zeros((n,n),dtype = float)
    pairs = np.empty((n*n,2),dtype=object)
    m = 0
    #loop through all possilbe combintions of stocks ie 3!=6
    for i in range(0,n):
        for j in range(0,n):
            S1 = data[:,i+1]
            print(S1)
            S2 = data[:,j+1]
            print(S2)
            result = ts.coint(S1,S2)
            pvalue_matrix[i,j] = result[1]
            #storing all paris in matrix form
            if (result[1] < 0.05) & (i!=j):
                pairs[m,0] = stocks[i]
                pairs[m,1] = stocks[j] 
                m = m + 1
    return pvalue_matrix,  pairs

In [5]:
#Example 2:
#SPY: SPDR S&P 500 ETF Trust
#DIA: SPDR Dow Jones Industrial Average ETF Trust

#Example 1:
#AAPL: Apple
#MSFT: Microsoft

today = datetime.date.today()
sp_list1 = ['SPY','DIA']
offset = max(1, (today.weekday() + 6) % 7 - 3)
timed = timedelta(offset)
today_business = today - timed
print("d1 =", today_business)
today = today_business.strftime("%Y-%m-%d")
start = '2010-01-01' 
end = today
print('S&P500 Stock download')
spy = yf.download(sp_list1, start, end)
#print(sp_list[0])
#print(spy)

today = datetime.date.today()
sp_list2 = ['AAPL','MSFT']
offset = max(1, (today.weekday() + 6) % 7 - 3)
timed = timedelta(offset)
today_business = today - timed
print("d1 =", today_business)
today = today_business.strftime("%Y-%m-%d")
start = '2010-01-01' 
end = today
print('S&P500 Stock download')
aapl = yf.download(sp_list2, start, end)

d1 = 2022-05-13
S&P500 Stock download
[*********************100%***********************]  2 of 2 completed
d1 = 2022-05-13
S&P500 Stock download
[*********************100%***********************]  2 of 2 completed


In [15]:
#cointegrated_pairs(spy)

#scipy.stats.pearsonr(S1,S2)

print(spy)

print('Edited Table:')

print(spy[:2])

#example = [[1, 2, 3],[4, 5, 6],[7, 8, 9]];

#print(example)

#print(example[2:])

#print(S1[:,5])

#scipy.stats.pearsonr(spy,spy)

             Adj Close                   Close                    High  \
                   DIA         SPY         DIA         SPY         DIA   
Date                                                                     
2010-01-04   80.143364   89.610832  105.660004  113.330002  105.870003   
2010-01-05   80.014343   89.848045  105.489998  113.629997  105.669998   
2010-01-06   80.067490   89.911331  105.559998  113.709999  105.849998   
2010-01-07   80.355682   90.290848  105.940002  114.190002  106.019997   
2010-01-08   80.484665   90.591309  106.110001  114.570000  106.110001   
...                ...         ...         ...         ...         ...   
2022-05-06  329.070007  411.339996  329.070007  411.339996  330.649994   
2022-05-09  322.600006  398.170013  322.600006  398.170013  326.369995   
2022-05-10  321.829987  399.089996  321.829987  399.089996  327.609985   
2022-05-11  318.459991  392.750000  318.459991  392.750000  326.070007   
2022-05-12  317.640015  392.339996  31

In [44]:
result = ts.coint(S1,S2)
result[1]

NameError: name 'S1' is not defined

In [42]:
def PnL(signals):
    signals_np = np.array(signals) # converts our pandas df into numpy array
    nu_shift = np.array(signals['nu'].shift(periods=1)) #lag timeseries by 1 trading day
    vol_shift = np.array(signals['volatility'].shift(periods=1)) #lag timeseries by 1 trading day
    negvol_shift = np.array(signals['negvolatility'].shift(periods=1)) #lag timeseries by 1 trading day
    vec_sig = np.zeros(len(signals))
    for i in range(len(signals)-1):
        #innovation > volatilty & innovation+1 < volatilty+1
        if (signals_np[i,0] > signals_np[i,1]): # t1 & t0 SHORT asset when current t changes to nu > volatlity
            vec_sig[i] = -1
        elif(signals_np[i,0] < signals_np[i,2]): #LONG assets when current t changes to nu < negvol
            vec_sig[i] = 1
        else:
            vec_sig[i] = 0
    # at the current day we enter our position
    vec_sig = pd.DataFrame({'vec_sig':vec_sig})
    return vec_sig

In [43]:
def backtesting(vec_sig,S1,S2):
    num_shares_y = vec_sig['vec_sig'].iloc[1:] * 1000
    num_shares_x = vec_sig['vec_sig'].iloc[1:] * -1000
    #forward fill x & y postion 
    PLX = num_shares_x.values * S1.pct_change().iloc[1:].values # difference of column X 
    PLY = (num_shares_y.values)  * S2.pct_change().iloc[1:].values #difference of column y 
    profit_loss = PLX + PLY
buy_hold_x = 1000
    buy_hold_y = 1000
    buy_hold_x = buy_hold_x *  S1.pct_change().iloc[1:] # 
    buy_hold_y = buy_hold_y *  S2.pct_change().iloc[1:]
    buy_hold   = buy_hold_x + buy_hold_y    
    buy_hold = sum(buy_hold)
    profit_loss = sum(profit_loss)
return profit_loss , buy_hold

IndentationError: unexpected indent (1288110750.py, line 9)